# Backfill notebook

In [ ]:
!pip install -U hopsworks --quiet

#### Data: Data comes in three different parquet files: 
- `credit_cards.parquet`: Credit card information such as expiration date and provider.
- `transactions.parquet`: Transaction information such as timestamp of transaction, the location of transaction, and the amount of transaction. Importantly, a binary **'fraud_label'** variable tells us whether a transaction was fraudulent or not.
- `profiles.parquet`: Credit card user information such as birthdate and city of residence.

All of these files have `cc_num` column as a natural join key.

#### Data Source: 
As real time sources of credit card fraud data are NOT easily accessible(owing to both, their financial value and associated security concerns), we conceptualize these three files as originating from separate data sources

### Imports

In [41]:
import pandas as pd
from datetime import datetime
import hopsworks

In [42]:
url = "https://repo.hops.works/master/hopsworks-tutorials/data/card_fraud_data"
credit_cards_df = pd.read_parquet(url + "/credit_cards.parquet")
credit_cards_df.head(5)

,cc_num,provider,expires
0,4796807885357879,visa,05/23
1,4529266636192966,visa,03/22
2,4922690008243953,visa,02/27
3,4897369589533543,visa,04/22
4,4848518335893425,visa,10/26


In [43]:
profiles_df = pd.read_parquet(url + "/profiles.parquet")
profiles_df.head(5)

,name,sex,mail,birthdate,City,Country,cc_num
0,Catherine Zimmerman,F,valenciajason@hotmail.com,1988-09-20,Bryn Mawr-Skyway,US,4796807885357879
1,Michael Williams,M,brettkennedy@yahoo.com,1977-03-01,Gates-North Gates,US,4529266636192966
2,Jessica Krueger,F,marthacruz@hotmail.com,1947-09-10,Greenfield,US,4922690008243953
3,Ruth Harris,F,james11@yahoo.com,1983-12-27,New City,US,4897369589533543
4,Paul Ashley,M,matthew97@hotmail.com,1974-11-10,Peabody,US,4848518335893425


In [44]:
trans_df = pd.read_parquet(url + "/transactions.parquet")
trans_df.head(3)

,tid,datetime,cc_num,category,amount,latitude,longitude,city,country,fraud_label
0,11df919988c134d97bbff2678eb68e22,2022-01-01 00:00:24,4473593503484549,Health/Beauty,62.95,42.30865,-83.48216,Canton,US,0
1,dd0b2d6d4266ccd3bf05bc2ea91cf180,2022-01-01 00:00:56,4272465718946864,Grocery,85.45,33.52253,-117.70755,Laguna Niguel,US,0
2,e627f5d9a9739833bd52d2da51761fc3,2022-01-01 00:02:32,4104216579248948,Domestic Transport,21.63,37.60876,-77.37331,Mechanicsville,US,0


## Feature Engineering
Fraudulent transactions can differ from regular ones in many different ways. Typical red flags would for instance be a large transaction volume/frequency in the span of a few hours. It could also be the case that elderly people in particular are targeted by fraudsters. To facilitate model learning we will create additional features based on these patterns. In particular, we will create two types of features:
1. **Features that aggregate data from different data sources**. This could for instance be the age of a customer at the time of a transaction, which combines the `birthdate` feature from `profiles.csv` with the `datetime` feature from `transactions.csv`.
2. **Features that aggregate data from multiple time steps**. An example of this could be the transaction frequency of a credit card in the span of a few hours, which is computed using a window function.

Let's start with the first category, aggregating from different sources.

#### Here we create three additional features. 'card_owner_age', 'expiry_days', and 'activity_level'.

In [45]:
from helper import features
import warnings
warnings.filterwarnings('ignore')

fraud_labels = trans_df[["tid", "cc_num", "datetime", "fraud_label"]]
fraud_labels.datetime = fraud_labels.datetime.map(lambda x: features.date_to_timestamp(x))

trans_df = trans_df.drop(['fraud_label'], axis=1)
trans_df = features.card_owner_age(trans_df, profiles_df)
trans_df = features.expiry_days(trans_df, credit_cards_df)
trans_df = features.activity_level(trans_df, 1)

trans_df

,tid,datetime,cc_num,category,amount,city,country,age_at_transaction,days_until_card_expires,loc_delta_t_minus_1,time_delta_t_minus_1
0,11df919988c134d97bbff2678eb68e22,1640995224000,4473593503484549,Health/Beauty,62.95,Canton,US,97.513297,1460.999722,0.221947,1.361921
1,dd0b2d6d4266ccd3bf05bc2ea91cf180,1640995256000,4272465718946864,Grocery,85.45,Laguna Niguel,US,33.752919,1733.999352,0.103615,1.050301
2,e627f5d9a9739833bd52d2da51761fc3,1640995352000,4104216579248948,Domestic Transport,21.63,Mechanicsville,US,80.899681,242.998241,0.167885,3.326192
3,6fb3e6beafbb92b8e15827037f603c52,1640995404000,4814447237003448,Health/Beauty,54.71,Nampa,US,53.526088,150.997639,0.263511,1.709850
4,be0b8acc57bfe126a5a392fd99e6ddd1,1640995435000,4515188652242507,Grocery,59.22,Pottstown,US,46.005059,515.997280,0.573918,2.961331
...,...,...,...,...,...,...,...,...,...,...,...
61905,e967e55653d02d9863ece69eb8068368,1651488694000,4581222106011785,Cash Withdrawal,97.49,Carlsbad,US,21.841523,-62.452477,0.000000,0.000000
62165,b03d8b31c07fff8378edf6f4b15faa6a,1651693447000,4176661673840251,Cash Withdrawal,43.65,Aloha,US,92.466847,180.177697,0.000000,0.000000
70175,3acb0c896855087ca5e0045ddee1121d,1651790995000,4295947736465426,Cash Withdrawal,100.92,Gainesville,US,53.123476,1487.048669,0.000000,0.000000
103887,44ae59516c9c0a6d43f75ba1df294e33,1651892153000,4697434187093357,Cash Withdrawal,99.91,Columbia Heights,US,94.778461,85.877859,0.000000,0.000000


In [38]:
# import sys
# sys.path.append('/content/drive/MyDrive/Colab Notebooks/helper/features.py')

We also need to create feature groups(a feature group can be seen as a collection of conceptually related features) in our feature store.
Let's create our first feature group.

In [46]:
project = hopsworks.login()
fs = project.get_feature_store()

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated

Paste it here: ··········
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/3342
Connected. Call `.close()` to terminate connection gracefully.


In [47]:
trans_fg = fs.get_or_create_feature_group(
    name="cc_trans_fraud",
    version=2,
    description="Credit Card transactions",
    primary_key=["cc_num"],
    event_time="datetime"
)

In [48]:
trans_fg.insert(trans_df, write_options={"wait_for_job" : False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/3342/fs/3289/fg/6524


Uploading Dataframe: 0.00% |          | Rows 0/106020 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/3342/jobs/named/cc_trans_fraud_2_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7fd55cad9f10>, None)

In [49]:
feature_descriptions = [
    {"name": "tid", "description": "Transaction id"},
    {"name": "datetime", "description": "Transaction time"},
    {"name": "cc_num", "description": "Number of the credit card performing the transaction"},
    {"name": "category", "description": "Expense category"},
    {"name": "amount", "description": "Dollar amount of the transaction"},
    {"name": "city", "description": "City in which the transaction was made"},
    {"name": "country", "description": "Country in which the transaction was made"},
    {"name": "age_at_transaction", "description": "Age of the card holder when the transaction was made"},
    {"name": "days_until_card_expires", "description": "Card validity days left when the transaction was made"},
    {"name": "loc_delta_t_minus_1", "description": "Haversine distance between this transaction location and the previous transaction location from the same card"},
    {"name": "time_delta_t_minus_1", "description": "Time in days between this transaction and the previous transaction location from the same card"},
]

for desc in feature_descriptions: 
    trans_fg.update_feature_description(desc["name"], desc["description"])

#### Now for the second part, we create features from each credit card aggregated from multiple time steps.


In [51]:
window_len = 4
window_aggs_df = features.aggregate_activity_by_hour(trans_df, window_len)
window_aggs_df.tail()

,trans_volume_mstd,trans_volume_mavg,trans_freq,loc_delta_mavg,cc_num,datetime
106015,2420.3075,2420.3075,2420.3075,0.045692,4032019521897961,1648119422000
106016,123.8425,123.8425,123.8425,0.034348,4032019521897961,1648468622000
106017,118.4400,118.4400,118.4400,0.034406,4032019521897961,1648817822000
106018,155.5125,155.5125,155.5125,0.034470,4032019521897961,1649167022000
106019,203.4075,203.4075,203.4075,0.034466,4032019521897961,1649516222000


In [52]:
window_aggs_fg = fs.get_or_create_feature_group(
    name=f"cc_trans_fraud_{window_len}h",
    version=2,
    description=f"Counts of the number of credit card transactions over {window_len} hour windows.",
    primary_key=["cc_num"],
    event_time="datetime"
)

In [53]:
window_aggs_fg.insert(window_aggs_df, write_options={"wait_for_job" : False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/3342/fs/3289/fg/6525


Uploading Dataframe: 0.00% |          | Rows 0/106020 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/3342/jobs/named/cc_trans_fraud_4h_2_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7fd55cf2ec50>, None)

In [54]:
feature_descriptions = [
    {"name": "datetime", "description": "Transaction time"},
    {"name": "cc_num", "description": "Number of the credit card performing the transaction"},
    {"name": "loc_delta_mavg", "description": "Moving average of location difference between consecutive transactions from the same card"},
    {"name": "trans_freq", "description": "Moving average of transaction frequency from the same card"},
    {"name": "trans_volume_mavg", "description": "Moving average of transaction volume from the same card"},
    {"name": "trans_volume_mstd", "description": "Moving standard deviation of transaction volume from the same card"},
]

for desc in feature_descriptions: 
    window_aggs_fg.update_feature_description(desc["name"], desc["description"])

#### Creating another fetaure group for fraud transactions

In [55]:
trans_label_fg = fs.get_or_create_feature_group(
    name="transactions_fraud_label",
    version=2,
    description="CC transactions that have been flagged as fraud",
    primary_key=['cc_num'],
    event_time='datetime'
)

trans_label_fg.insert(fraud_labels, write_options={"wait_for_job" : False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/3342/fs/3289/fg/6526


Uploading Dataframe: 0.00% |          | Rows 0/106020 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/3342/jobs/named/transactions_fraud_label_2_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7fd55d0a4bd0>, None)

In [56]:
feature_descriptions = [
    {"name": "tid", "description": "Transaction id"},
    {"name": "cc_num", "description": "Number of the credit card performing the transaction"},    
    {"name": "datetime", "description": "Transaction time"},
    {"name": "fraud_label", "description": "Whether the transaction was fraudulent or not"},
]
for desc in feature_descriptions: 
    trans_label_fg.update_feature_description(desc["name"], desc["description"])